In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [76]:
from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil, pi

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool, \
DataRange1d, DatetimeTickFormatter
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap, cumsum
from bokeh.io import show, output_notebook
from bokeh import palettes

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [3]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [10]:
path = "/home/wen/Documentos/Proyecto/Datos Agrosoft/CTA_CTE.TPS"
df = parser(path);df

,tipo_movim,cod_proveedor,fecha_comprob,nro_comprob,importe,periodo_contable,prefijo,tipo_comprobante
0,1,34,2011-11-24,241111,520.00,2011-11-24,0,NaN
1,1,34,2011-11-24,241111,36.15,2011-11-24,0,NaN
2,1,34,2011-10-13,131011,47.24,2011-10-13,0,NaN
3,1,35,2011-12-30,934,5327.26,2011-12-30,0,NaN
4,1,35,2011-12-30,236,11381.00,2011-12-30,0,NaN
...,...,...,...,...,...,...,...,...
7956,1,21,2011-08-15,250,2383.70,2011-11-30,0,NaN
7957,4,21,2011-08-15,16,2530.11,2011-11-30,0,NaN
7958,1,22,2011-10-31,23918,68.78,2011-11-30,0,NaN
7959,1,23,2011-11-30,58832,1203.05,2011-11-30,0,NaN


In [84]:
from app.models import CtaCte
proveedor = CtaCte.objects.values(
    'tipo_movim__descripcion',
    'cod_proveedor__razon_social',
    'cod_proveedor__direccion',
    'cod_proveedor__localidad__descripcion',
    'cod_proveedor__cuit',
    'cod_proveedor__saldo_cta_cte',
    'fecha_comprob',
    'nro_comprob',
    'importe',
    'periodo_contable',
    'tipo_comprobante') \
    .order_by('fecha_comprob')

df = DataFrame(proveedor)
df.rename(columns={'cod_proveedor__cuit': 'cuit',
                   'cod_proveedor__direccion': 'direccion',
                   'cod_proveedor__localidad__descripcion': 'localidad',
                   'cod_proveedor__razon_social': 'razon_social',
                   'tipo_movim__descripcion': 'tipo_movimiento',
                   'cod_proveedor__saldo_cta_cte': 'saldo'},
          inplace=True)
df['fecha_comprob'] = to_datetime(df.fecha_comprob)

In [85]:
df = df.loc[df['razon_social'].str.contains('Bonafede Emiliana', case=False, na=False,
                                           regex=False)].copy()

In [88]:
df

,tipo_movimiento,razon_social,direccion,localidad,cuit,saldo,fecha_comprob,nro_comprob,importe,periodo_contable,tipo_comprobante
4409,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-28,0,5000.0,2015-01-28,None
4417,Orden de Pago,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-30,2533,8000.0,None,None
4419,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-01-30,1,3000.0,2015-01-30,None
4424,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-02-01,2,3000.0,2015-02-01,None
4463,Orden de Pago,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-02-10,2703,3000.0,None,None
4544,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-03-01,3,3000.0,2015-03-01,None
4568,Orden de Pago,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-03-10,2704,3000.0,None,None
4684,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-04-01,4,3000.0,2015-04-01,None
4730,Orden de Pago,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-04-10,2705,3000.0,None,None
4817,Factura Cuenta Corriente,Bonafede Emiliana (Sueldo),25 De Mayo 201,Colonia Aldao,27111111111,-38000.0,2015-05-01,5,3000.0,2015-05-01,None


In [90]:
gb = DataFrame(df.groupby('tipo_movimiento').importe.sum())

In [69]:
df_debe = df[df.tipo_movimiento == 'Orden de Pago'].copy()
df_haber = df[df.tipo_movimiento == 'Factura Cuenta Corriente'].copy()

In [92]:
gb['angle'] = gb.importe / gb.importe.sum() * 2 * pi
source = ColumnDataSource(gb)

tooltips = [("Tipo Movimiento", "@tipo_movimiento"),
            ("Importe", "@importe{$0,0.00}")]
hovertool = HoverTool(names=['wedge'], tooltips=tooltips)
plot = figure(
    tools=["tap,reset,save", hovertool],
    toolbar_location='above')
plot.wedge(x=0, y=0, radius=0.4,
           start_angle=cumsum('angle', include_zero=True),
           end_angle=cumsum('angle'),
           line_color='white', line_width=0.5,
           legend='tipo_movimiento',
           source=source, name='wedge')

show(plot)